In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense
from boruta import BorutaPy

In [2]:
data = pd.read_csv("Data_processed.csv")

In [3]:
data[:5]

,X,Date,Zone1Position,Zone2Position,Zone3Position,SKU,Zone1_Row_Num,Zone1_Col_Num,Zone2_Row_Num,Zone2_Col_num,...,Zone2_Humidity_Max,Zone2_Humidity_Range,Zone3_Humidity_Avg,Zone3_Humidity_Min,Zone3_Humidity_Max,Zone3_Humidity_Range,Block_Num,Block_Position,Block_Orientation,Result_Type
0,1,09/01/2020,3.0,2.0,3.0,A001,1.0,3.0,1.0,2.0,...,61.80,10.0,39.19,33.19,45.19,12.0,1000.0,1.0,1,Defect_1
1,2,03/12/2019,8.0,4.0,3.0,A001,2.0,4.0,2.0,2.0,...,58.12,16.0,48.50,40.50,56.50,16.0,1003.0,1.0,1,Defect_1
2,3,21/11/2019,4.0,2.0,3.0,B003,1.0,4.0,1.0,2.0,...,50.97,10.0,52.20,46.20,58.20,12.0,1001.0,4.0,1,PASS
3,4,22/11/2019,6.0,3.0,1.0,B003,2.0,2.0,2.0,1.0,...,59.13,4.0,39.29,31.29,47.29,16.0,1003.0,2.0,1,PASS
4,5,23/12/2019,3.0,2.0,3.0,B003,1.0,3.0,1.0,2.0,...,65.80,18.0,45.37,42.37,48.37,6.0,1002.0,1.0,1,PASS


In [4]:
data["Result_binary"] = np.where(data['Result_Type']== "PASS", 
                                 '0', '1')
data["Result_binary"].describe()

count     75750
unique        2
top           0
freq      54720
Name: Result_binary, dtype: object

In [44]:
data_model=data[["Zone1Position", "Zone2Position", "Zone3Position", "SKU", "Block_Num", "Block_Position", 
                "Result_Type", "Zone3_Humidity_Avg", "Zone2_Humidity_Avg", "Zone1_Humidity_Avg","Zone3_Temp_Avg",
               "Zone2_Temp_Avg", "Zone1_Temp_Avg", "Zone1_Dur", "Zone2_Dur", "Zone3_Dur"]]

data_model2=data[["Zone1Position", "Zone2Position", "Zone3Position", "SKU", "Block_Num", "Block_Position", 
                "Result_Type", "Zone3_Humidity_Range", "Zone2_Humidity_Range", "Zone1_Humidity_Range",
                  "Zone3_Temp_Range","Zone2_Temp_Range", "Zone1_Temp_Range", "Zone1_Dur", "Zone2_Dur", "Zone3_Dur"]]

#the choice of features for data_model2 is based on the results of the multinomial regression models (see the R script)

data_model3=data[["Result_Type", "Zone1Position", "Zone2Position", "SKU", "Block_Num", "Block_Position", "Zone1_Area", 
                  "Zone1_In_Zone3_Out_Dur", "Zone1_Col_Num","Zone2_Col_num", "Zone3_Humidity_Max",
                  "Zone3_Humidity_Min", "Zone3_Humidity_Avg", "Zone2_Humidity_Max", "Zone2_Temp_Max"]]

In [45]:
train, test = train_test_split(data_model3.dropna(), test_size = 0.2)

1. Decision Tree

In [46]:
dt = tree.DecisionTreeClassifier(criterion='entropy')
x_train = pd.get_dummies(train[["Zone1Position", "Zone2Position", "SKU", "Block_Num", "Block_Position", "Zone1_Area",
                  "Zone1_In_Zone3_Out_Dur", "Zone1_Col_Num","Zone2_Col_num", "Zone3_Humidity_Max", 
                  "Zone3_Humidity_Min", "Zone3_Humidity_Avg", "Zone2_Humidity_Max", "Zone2_Temp_Max"]], 
                         drop_first=True)
y_train = train["Result_Type"]
dt = dt.fit(x_train, y_train)
x_test = pd.get_dummies(test[["Zone1Position", "Zone2Position", "SKU", "Block_Num", "Block_Position", "Zone1_Area",
                  "Zone1_In_Zone3_Out_Dur", "Zone1_Col_Num","Zone2_Col_num", "Zone3_Humidity_Max", 
                  "Zone3_Humidity_Min", "Zone3_Humidity_Avg", "Zone2_Humidity_Max", "Zone2_Temp_Max"]], 
                        drop_first=True)
y_test = test["Result_Type"]
y_pred = dt.predict(x_test)
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,Defect_1,Defect_2,Defect_3,Defect_4,Defect_5,PASS,All
True,,,,,,,
Defect_1,272,4,24,27,24,425,776
Defect_2,8,671,0,5,6,141,831
Defect_3,48,1,1156,6,8,151,1370
Defect_4,23,13,7,52,36,521,652
Defect_5,24,3,11,17,44,445,544
PASS,468,127,186,618,579,8952,10930
All,843,819,1384,725,697,10635,15103


In [20]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Defect_1       0.34      0.37      0.35       807
    Defect_2       0.75      0.81      0.78       772
    Defect_3       0.82      0.85      0.83      1317
    Defect_4       0.08      0.08      0.08       665
    Defect_5       0.06      0.07      0.07       613
        PASS       0.84      0.81      0.82     10929

    accuracy                           0.73     15103
   macro avg       0.48      0.50      0.49     15103
weighted avg       0.74      0.73      0.73     15103



In [48]:
clf=RandomForestClassifier(n_estimators=100)
clf.fit(x_train,y_train)

y_pred=clf.predict(x_test)

In [49]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8500960074157452


In [50]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Defect_1       0.46      0.23      0.31       776
    Defect_2       0.96      0.86      0.91       831
    Defect_3       0.93      0.93      0.93      1370
    Defect_4       0.92      0.02      0.03       652
    Defect_5       0.91      0.02      0.04       544
        PASS       0.85      0.97      0.91     10930

    accuracy                           0.85     15103
   macro avg       0.84      0.51      0.52     15103
weighted avg       0.85      0.85      0.81     15103



In [64]:
importances = clf.feature_importances_
importances

array([0.04320176, 0.04273265, 0.08412487, 0.06831236, 0.19312848,
       0.02120965, 0.01049976, 0.08816993, 0.0817535 , 0.06452484,
       0.09587279, 0.09243264, 0.01100791, 0.04824784, 0.00928548,
       0.01176271, 0.00247021, 0.02872942, 0.00253322])

In [53]:
from treeinterpreter import treeinterpreter as ti
prediction, bias, contributions = ti.predict(clf, x_test)



<b> 2. Neural network </b> 

In [12]:
min_max_scaler = preprocessing.MinMaxScaler()
x_train_scale = min_max_scaler.fit_transform(x_train)
x_test_scale = min_max_scaler.fit_transform(x_test)
x_train_scale

array([[0.42857143, 0.33333333, 0.2       , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.28571429, 0.33333333, 0.2       , ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.14285714, 0.        , 0.2       , ..., 1.        , 0.        ,
        0.        ],
       [1.        , 1.        , 0.4       , ..., 1.        , 0.        ,
        0.        ],
       [0.28571429, 0.33333333, 1.        , ..., 0.        , 0.        ,
        0.        ]])

2. Logistic regression

In [51]:
data_model_logit=data[["Zone1Position", "Zone2Position", "Zone3Position", "SKU", "Block_Num", "Block_Position",
                       "Result_binary", "Zone3_Humidity_Avg", "Zone2_Humidity_Avg", "Zone1_Humidity_Avg",
                       "Zone3_Temp_Avg","Zone2_Temp_Avg", "Zone1_Temp_Avg", "Zone1_Dur", "Zone2_Dur", "Zone3_Dur"]]

In [ ]:
from sklearn.preprocessing import StandardScaler
train_data_lr, test_data_lr = train_test_split(data_model_logit, test_size = 0.2)
scaler = StandardScaler()
x_train_lr = scaler.fit_transform(pd.get_dummies(train_data_lr[["Zone1Position", "Zone2Position", "Zone3Position", "SKU", "Block_Num", 
                                     "Block_Position","Zone3_Humidity_Avg", "Zone2_Humidity_Avg", "Zone1_Humidity_Avg",
                                     "Zone3_Temp_Avg","Zone2_Temp_Avg", "Zone1_Temp_Avg", "Zone1_Dur", "Zone2_Dur", 
                                     "Zone3_Dur"]],drop_first=True))
                           
y_train_lr = train_data_lr["Result_binary"]
lr = LogisticRegression(random_state=0).fit(x_train_lr, y_train_lr)
x_test_lr = scaler.fit_transform(pd.get_dummies(test_data_lr[["Zone1Position", "Zone2Position", "Zone3Position", "SKU", "Block_Num", 
                                     "Block_Position","Zone3_Humidity_Avg", "Zone2_Humidity_Avg", "Zone1_Humidity_Avg",
                                     "Zone3_Temp_Avg","Zone2_Temp_Avg", "Zone1_Temp_Avg", "Zone1_Dur", "Zone2_Dur", 
                                     "Zone3_Dur"]],drop_first=True))
y_test_lr = test_data_lr["Result_binary"]
y_pred_lr = lr.predict(X_test_lr)
print(classification_report(y_test_lr, y_pred_lr))